In [2]:
import re
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
# Read .csv data.
data = pd.read_csv("../data/raw/parlspeech_bundestag.csv", parse_dates=['date'], low_memory=False)
data = data.reset_index()

In [4]:
# Discard entries where "chair" variable is True. These are not speeches by MPs.
data = data[data['chair']==False]
data.iloc[0].text[1000:1500]

"harmlost und den Bürgern mögliche Belastungen verschwiegen . . _ Ich wußte gar nicht , daß Sie in der Früh' schon so lebendig sind . _ (Struck [SPD] : Sie werden sich noch wundern , Herr Waigel ! _ Wieczorek [Duisburg] [SPD] : Wir sind doch keine Schlafmützen !) Das Gegenteil , meine Damen und Herren , läßt sich beweisen . . Ich habe in einer Ansprache vor dem Zentralausschuß der Deutschen Landwirtschaft am 14 . Februar 1990 folgendes gesagt : (Vogel [SPD] : Steuern werden nicht erhöht ! ) Die w"

In [5]:
data.iloc[0].text[-500:]

'r Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . . Die Infrastruktur muß praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen . (Anhaltende Zurufe des Abg Duve [SPD]) _Frau Präsidentin , da muß ein Blähhals am Werke sein . .'

In [6]:
data.iloc[-1].text[-500:]

'Ich bedanke mich bei den Mitarbeitern der Verwaltung, beim Stenografischen Dienst und bei der Saalassistenz. Und jetzt haben Sie das letzte Wort. Ich bedanke mich und freue mich auf das nächste Jahr. Vielen Dank. – Bürgermeister überziehen halt etwas öfter; aber, Herr Präsident, Sie haben es im Griff.'

In [7]:
data.iloc[-3].text[-500:]

'skutieren können. Es gibt auch Punkte, die ich gern ablehne. Aber ich denke, wir sind hier auf dem richtigen Weg, das Thema Fachkräfte entsprechend voranzutreiben, auch im sozialen Bereich. Abschließend wünsche ich Ihnen allen schöne Weihnachten und besinnliche Feiertage; denn Zeit zur Besinnung und Einsicht würde dem einen oder anderen nicht schaden. Zudem bedanke ich mich bei dieser Gelegenheit für in der Regel fair geführte Debatten in diesem Jahr – in der Regel – sowie beim Tagungspräsidium.'

In [8]:
data.iloc[-3].text[1000:1500]

'en. Wir haben gehört: Auch die Fallzahlen in der Kinder- und Jugendhilfe sind gestiegen. Im Koalitionsvertrag stehen noch viele andere gute Dinge. Das zeigt natürlich, dass wir Fachkräfte benötigen. Ich selber war Bürgermeister. In dieser Zeit habe ich gemerkt: Wenn man engagiert ist und qualifiziertes Personal haben will, ist es sehr schwierig. Dennoch sollte man einige sachliche Punkte erwähnen. Während sich die Zahl der zu betreuenden Kinder zwischen 2008 und 2017 um 16 Prozent erhöht hat, ha'

We notice some immediate problems with the data:
1. It contains comments from other members of parliament within parentheses
2. The old speeches use the old German spelling rules
3. The old speeches sometimes use underscores ('_'). This was probably done for formatting purposes, but we do not need it.
4. This is probably not a problem at all, but earlier speeches use spaces between punctuation marks and newer speeches do not.

First, let's fix 1. by simply removing all text inside parentheses.

In [9]:
# Remove bracketed content from speeches. This content is commentary; not part of the original speech.
def remove_brackets(text):
    text_clean = re.sub(r'\([^)]*\)',"",text)
    text_clean = re.sub(r'\([^)]*',"",text_clean) # Sometimes there will be a comment right before the end of the speech with no closing paren
    text_clean = re.sub("\[INTERVENTION BEGINS\]", "", text_clean)
    text_clean = re.sub("\[INTERVENTION ENDS\]", "", text_clean)
    return text_clean

data['text_clean'] = data['text'].apply(remove_brackets)

In [10]:
data.iloc[0].text_clean[-500:]

'schäden erbringen .  _Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . . Die Infrastruktur muß praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  _Frau Präsidentin , da muß ein Blähhals am Werke sein . .'

In [11]:
from collections import Counter
full_str = data['text'].str.cat()
counter = Counter(full_str)
counter

Counter({'F': 1228008,
         'r': 37206558,
         'a': 26895519,
         'u': 20333979,
         ' ': 97542920,
         'P': 1222709,
         'ä': 2845498,
         's': 31918591,
         'i': 42487418,
         'd': 23891862,
         'e': 83279537,
         'n': 54488238,
         't': 30625488,
         '!': 255575,
         'M': 1507505,
         'h': 22641344,
         'g': 15721770,
         'D': 2765248,
         'm': 11124296,
         'H': 1066827,
         'I': 1436284,
         'z': 6000252,
         'B': 2022014,
         'v': 3468544,
         'o': 11887804,
         'l': 17384350,
         'E': 1760080,
         'w': 7141498,
         'f': 7201117,
         '1': 422216,
         '9': 252407,
         'W': 1584372,
         '.': 5395340,
         'V': 1094263,
         'A': 2105221,
         'b': 8625784,
         'ü': 3406054,
         '4': 122301,
         '0': 677643,
         'c': 14588996,
         '3': 141151,
         'O': 271465,
         'k': 5641104,
  

In [12]:
search_text = data[data['text_clean'].str.find('[') > -1].iloc[3].text_clean
get_inside_brackets = lambda x: re.findall(r'\[[^\]]*\]', x)
broken_candidates = data[data['text_clean'].str.find('[') > -1]
broken_candidates

,index,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,text_clean
9062,9062,1991-10-30,NaN,26,Horst Peter,SPD,383.0,False,333,"Ich gestehe Ihnen zu , daß diese Ausgaben in d...",DE-Bundestag,DEU,"Ich gestehe Ihnen zu , daß diese Ausgaben in d..."
12358,12358,1991-12-12,NaN,15,Claudia Nolte,CDU/CSU,211.0,False,1476,Frau Präsidentin ! Meine sehr geehrten Damen u...,DE-Bundestag,DEU,Frau Präsidentin ! Meine sehr geehrten Damen u...
12655,12655,1991-12-13,NaN,25,Heidemarie Wieczorek-Zeul,SPD,383.0,False,275,"Ich möchte diesen Punkt erst zu Ende bringen ,...",DE-Bundestag,DEU,"Ich möchte diesen Punkt erst zu Ende bringen ,..."
13483,13483,1992-01-23,NaN,8,Hermann Bachmaier,SPD,383.0,False,101,"Ja , bitte . Heinrich L . Kolb (FDP) : Herr Ba...",DE-Bundestag,DEU,"Ja , bitte . Heinrich L . Kolb : Herr Bachmai..."
13835,13835,1992-01-24,NaN,32,Iris Gleicke,SPD,383.0,False,1275,Herr Präsident ! Meine sehr verehrten Kollegin...,DE-Bundestag,DEU,Herr Präsident ! Meine sehr verehrten Kollegin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
370789,370789,2018-07-03,Tagesordnungspunkt I.8: Einzelplan 17 Bundesm...,192,Beatrix von Storch,AfD,1976.0,False,698,Frau Präsidentin! Sehr geehrte Damen und Herre...,DE-Bundestag,DEU,Frau Präsidentin! Sehr geehrte Damen und Herre...
371760,371760,2018-09-12,punkte 1 a und 1 b – fort: a) Erste Beratung d...,84,Simone Barrientos,PDS/LINKE,1545.0,False,474,Sehr geehrter Herr Präsident! Werte Kolleginne...,DE-Bundestag,DEU,Sehr geehrter Herr Präsident! Werte Kolleginne...
375958,375958,2018-11-08,Damit rufe ich die Tagesordnungspunkte 4 a bis...,26,Markus Kurth,GRUENE,1816.0,False,801,Herr Präsident! Liebe Kolleginnen und Kollegen...,DE-Bundestag,DEU,Herr Präsident! Liebe Kolleginnen und Kollegen...
376936,376936,2018-11-21,Tagesordnungspunkt I.9: hier: Einzelplan 04 Bu...,16,Sahra Wagenknecht,PDS/LINKE,1545.0,False,2436,Herr Präsident! Sehr geehrte Damen und Herren!...,DE-Bundestag,DEU,Herr Präsident! Sehr geehrte Damen und Herren!...


In [13]:
data[data.index == 376936].iloc[0].text[10200:10700]

'. Es ist doch bekannt, wie heute bei Ryanair, bei Amazon, bei der Deutschen Post und in vielen anderen Unternehmen mit Mitarbeitern umgesprungen wird. Und es sind Ihre Gesetze, die das möglich machen, die möglich machen, dass Menschen in schlecht bezahlten Jobs gedemütigt werden oder dass ömer[CDU/CSU]: sie als Leiharbeiter und Dauerbefristete der Willkür ihrer Arbeitgeber in besonderem Maße ausgeliefert sind. Es sind Ihre Gesetze, die möglich machen, dass Arbeitslose im Jobcenter schikaniert we'

In [14]:
data[data.index == 9062].iloc[0].text[:500]

'Ich gestehe Ihnen zu , daß diese Ausgaben in den letzten Jahren gesunken sind . . : Wie stark denn ? _ Klaus Kirschner [SPD] : Nachdem sie vorher hochgegangen sind !) _ Ich habe die Prozentzahlen jetzt nicht vor mir liegen . Ich lasse mich da gerne von Ihnen belehren . Die Senkung , die zu erwarten war , ist übrigens durch den Blüm-Bauch begründet , d . h . durch die Möglichkeit , sich noch vor Toresschluß den notwendigen Zahnersatz zu besorgen . Hinterher ist natürlich eine Nachfragelücke entst'

Some of the transcriptions simply have broken formatting for commentaries. We kick these candidates out entirely. In the process, we lose some data (a negligible amount in our opinion), but the data that remains is of higher quality.

In [15]:
indices_to_drop = broken_candidates.index
filtered_data = data.drop(indices_to_drop)
num_dropped = len(data) - len(filtered_data)
print(f"Dropped {num_dropped} ({100 * num_dropped / len(data):.2f}%) speeches. {len(filtered_data)} speeches remain.")
filtered_data[filtered_data['text_clean'].str.find('[') > -1]

Dropped 1079 (0.51%) speeches. 211381 speeches remain.


,index,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,text_clean


Next, let's remove the underscores.

In [16]:
filtered_data.iloc[0].text_clean[-500:]

'schäden erbringen .  _Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . . Die Infrastruktur muß praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  _Frau Präsidentin , da muß ein Blähhals am Werke sein . .'

In [17]:
def remove_underscores(text):
    return re.sub('_', '', text)

filtered_data['text_clean'] = filtered_data['text_clean'].apply(remove_underscores)

In [18]:
filtered_data.iloc[0].text_clean[-500:]

'ltschäden erbringen .  Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . . Die Infrastruktur muß praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  Frau Präsidentin , da muß ein Blähhals am Werke sein . .'

Finally, we tackle the issue of the German spelling reform. We looked for Python packages to handle this part for us and found nothing. Apparently Word can do it, and there is some really old software from Duden that can also do it. We also tried a spellcheck / autocorrect package, but this solution would destroy more than it would fix (e.g. it would automatically correct names like "Rolf" to "roll").
Thus, we do the bare minimum and transform 'ß' ("esszet") into 'ss'. Doing more would probably help the final result, but it's not feasible for the scope of this project.

In [19]:
def replace_esszet(text):
    return re.sub('ß', 'ss', text)

filtered_data['text_clean'] = filtered_data['text_clean'].apply(replace_esszet)

In [20]:
filtered_data.iloc[0].text_clean[-500:]

'schäden erbringen .  Ja , wenn das für Sie alles bekannt war , wie konnte dann ein so kluger Mann wie Ihr Kanzlerkandidat von einem guten Investitionsstandort sprechen ? Diesen Zwiespalt und Widerspruch in Ihrer Argumentation verstehe ich nicht . . Die Infrastruktur muss praktisch von Grund auf überholt werden . So ist das völlig überlastete Eisenbahnnetz in seiner Ausbauqualität sogar noch weit hinter den Stand von 1939 zurückgefallen .  Frau Präsidentin , da muss ein Blähhals am Werke sein . .'

In [21]:
"""
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stop_words = stopwords.words('german')

vectorizer = CountVectorizer(stop_words=stop_words) # max_df and min_df might be worth checking out later
vectorizer.fit(filtered_data['text_clean'])
len(vectorizer.vocabulary_)
"""

"\nfrom sklearn.feature_extraction.text import CountVectorizer\nfrom nltk.corpus import stopwords\nstop_words = stopwords.words('german')\n\nvectorizer = CountVectorizer(stop_words=stop_words) # max_df and min_df might be worth checking out later\nvectorizer.fit(filtered_data['text_clean'])\nlen(vectorizer.vocabulary_)\n"

In [22]:
import string

def is_word(s):
    return any(c.isalnum() for c in s)

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def get_avg_word_length(text):
    words = remove_punctuation(text).split()
    return sum([len(word) for word in words]) / len(words)

filtered_data['avg_word_length'] = filtered_data['text_clean'].apply(get_avg_word_length)
filtered_data

,index,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,text_clean,avg_word_length
0,0,1991-03-12,NaN,2,Theodor Waigel,CDU/CSU,211.0,False,880,Frau Präsidentin ! Meine sehr geehrten Damen u...,DE-Bundestag,DEU,Frau Präsidentin ! Meine sehr geehrten Damen u...,6.414523
2,2,1991-03-12,NaN,4,Theodor Waigel,CDU/CSU,211.0,False,5756,Nicht viel besser sieht es im Straßenwesen aus...,DE-Bundestag,DEU,Nicht viel besser sieht es im Strassenwesen au...,6.379921
4,4,1991-03-12,NaN,6,Barbara Höll,PDS/LINKE,86.0,False,233,Geehrte Frau Präsidentin ! Meine Damen und Her...,DE-Bundestag,DEU,Geehrte Frau Präsidentin ! Meine Damen und Her...,6.666667
6,6,1991-03-12,NaN,8,Jürgen Rüttgers,CDU/CSU,211.0,False,242,Frau Präsidentin ! Meine sehr verehrten Damen ...,DE-Bundestag,DEU,Frau Präsidentin ! Meine sehr verehrten Damen ...,5.441748
8,8,1991-03-12,NaN,10,Peter Struck,SPD,383.0,False,31,Frau Präsidentin ! Meine Damen und Herren ! Wi...,DE-Bundestag,DEU,Frau Präsidentin ! Meine Damen und Herren ! Wi...,6.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379535,379535,2018-12-14,Tagesordnungspunkt 23: Beratung des Antrags de...,186,Grigorios Aggelidis,FDP,573.0,False,648,Sehr geehrter Herr Präsident! Meine sehr geehr...,DE-Bundestag,DEU,Sehr geehrter Herr Präsident! Meine sehr geehr...,5.549383
379537,379537,2018-12-14,Tagesordnungspunkt 23: Beratung des Antrags de...,188,Katja Dörner,GRUENE,1816.0,False,688,Herr Präsident! Liebe Kolleginnen! Liebe Kolle...,DE-Bundestag,DEU,Herr Präsident! Liebe Kolleginnen! Liebe Kolle...,5.635174
379539,379539,2018-12-14,Tagesordnungspunkt 23: Beratung des Antrags de...,190,Michael Kießling,CDU/CSU,211.0,False,815,Sehr geehrter Herr Präsident! Liebe Kolleginne...,DE-Bundestag,DEU,Sehr geehrter Herr Präsident! Liebe Kolleginne...,5.403681
379541,379541,2018-12-14,Tagesordnungspunkt 23: Beratung des Antrags de...,192,Michael Kießling,CDU/CSU,211.0,False,9,Selbstverständlich. – Einen Satz würde ich ger...,DE-Bundestag,DEU,Selbstverständlich. – Einen Satz würde ich ger...,5.555556


In [23]:
filtered_data.sort_values(by=['avg_word_length'], ascending=False)[:20]

,index,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,text_clean,avg_word_length
26279,26279,1993-01-14,NaN,42,Arno Schmidt,FDP,573.0,False,2,Nichtregierungsorganisation !,DE-Bundestag,DEU,Nichtregierungsorganisation !,27.0
170169,170169,2003-07-03,NaN,131,Irmingard Schewe-Gerigk,GRUENE,1816.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
52936,52936,1994-09-21,NaN,410,Iris Gleicke,SPD,383.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
209956,209956,2007-03-01,NaN,67,Sigmar Gabriel,SPD,383.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
229346,229346,2008-09-18,NaN,96,Sigmar Gabriel,SPD,383.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
188147,188147,2005-01-20,NaN,175,Jürgen Koppelin,FDP,573.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
320018,320018,2014-11-27,Tagesordnungspunkt I.13: Einzelplan 30 Bundes...,138,Ewald Schurer,SPD,383.0,False,1,Selbstverständlich.,DE-Bundestag,DEU,Selbstverständlich.,18.0
370564,370564,2018-06-29,Tagesordnungspunkt 25: Beratung des Antrags d...,237,Sepp Müller,CDU/CSU,211.0,False,1,Selbstverständlich.,DE-Bundestag,DEU,Selbstverständlich.,18.0
93984,93984,1997-05-15,NaN,8,Kristin Heyne,GRUENE,1816.0,False,2,Selbstverständlich .,DE-Bundestag,DEU,Selbstverständlich .,18.0
370568,370568,2018-06-29,Tagesordnungspunkt 25: Beratung des Antrags d...,241,Sepp Müller,CDU/CSU,211.0,False,1,Selbstverständlich.,DE-Bundestag,DEU,Selbstverständlich.,18.0


In [24]:
from collections import Counter

def count_char(text, char):
    return text.count(char)

def count_exclamations(text):
    return count_char(text, '!') / len(text)

def count_questions(text):
    return count_char(text, '?') / len(text)

filtered_data['relative_num_exclamations'] = filtered_data.text_clean.apply(count_exclamations)
filtered_data['relative_num_questions'] = filtered_data.text_clean.apply(count_questions)

In [25]:
filtered_data.sort_values(by=['relative_num_questions'], ascending=False)[:1000]

,index,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,text_clean,avg_word_length,relative_num_exclamations,relative_num_questions
331551,331551,2015-09-24,Zusatzpunkt 3: 5a)Beratung des Antrags der Abg...,98,Petra Hinz,SPD,383.0,False,1,Ja?,DE-Bundestag,DEU,Ja?,2.000000,0.0,0.333333
361568,361568,2018-02-22,Tagesordnungspunkt 5: Beratung des Antrags de...,94,Philipp Amthor,CDU/CSU,211.0,False,1,Ja?,DE-Bundestag,DEU,Ja?,2.000000,0.0,0.333333
372800,372800,2018-09-27,Tagesordnungspunkt 6: Erste Beratung des von ...,104,Philipp Amthor,CDU/CSU,211.0,False,1,Ja?,DE-Bundestag,DEU,Ja?,2.000000,0.0,0.333333
288762,288762,2012-09-13,Tagesordnungspunkt 4 b: Beratung der Beschlus...,192,Mechthild Heil,CDU/CSU,211.0,False,1,Ja?,DE-Bundestag,DEU,Ja?,2.000000,0.0,0.333333
329697,329697,2015-06-19,Tagesordnungspunkte 34 a und 34 b: a) Beratun...,129,Hermann Färber,CDU/CSU,211.0,False,1,Ja?,DE-Bundestag,DEU,Ja?,2.000000,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34501,34501,1993-06-23,NaN,236,Jürgen Augustinowitz,CDU/CSU,211.0,False,24,"Eine kurze Zusatzfrage , Frau Staatssekretärin...",DE-Bundestag,DEU,"Eine kurze Zusatzfrage , Frau Staatssekretärin...",6.050000,0.0,0.013514
14491,14491,1992-02-19,NaN,15,Friedhelm Julius Beucher,SPD,383.0,False,11,"Sind Sie bereit , uns diese Namen gegebenenfal...",DE-Bundestag,DEU,"Sind Sie bereit , uns diese Namen gegebenenfal...",6.888889,0.0,0.013514
128090,128090,2000-02-16,NaN,128,Ulrike Flach,FDP,573.0,False,24,"Herr Staatssekretär , befürchten Sie nicht ein...",DE-Bundestag,DEU,"Herr Staatssekretär , befürchten Sie nicht ein...",6.100000,0.0,0.013423
117927,117927,1999-06-16,NaN,227,Ulrike Flach,FDP,573.0,False,24,Wie schätzen Sie die Auswirkungen auf den Arbe...,DE-Bundestag,DEU,Wie schätzen Sie die Auswirkungen auf den Arbe...,5.590909,0.0,0.013423


In [28]:
from nltk.corpus import stopwords
stop_words = stopwords.words('german')

def is_stop_word(word):
    return word in stop_words

def get_stop_word_fraction(text):
    words = remove_punctuation(text).lower().split()
    num_stop_words = len([word for word in words if is_stop_word(word)])
    return num_stop_words / len(words)

filtered_data['stop_word_fraction'] = filtered_data.text_clean.apply(get_stop_word_fraction)

In [30]:
from HanTa import HanoverTagger as ht
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [41]:
import nltk
sent = "Die Europawahl in den Niederlanden findet immer donnerstags statt."

words = nltk.word_tokenize(filtered_data.text_clean[0])
lemmata = tagger.tag_sent(words,taglevel= 1)
" ".join([lemma[1] for lemma in lemmata])

"Frau Präsidentin -- mein sehr geehrt Dame und Herr -- der ihnen heute zur erster Beratung vorliegend Entwurf des Bundeshaushalts 1991 sein der Haushalt der Wiedervereinigung -- fast ein Viertel aller Ausgabe von über 400 Milliarde Dm beziehen sich auf die am 3 -- Oktober 1990 neu hinzukommen Bundesland -- Umfang und Struktur dieses Haushaltsentwurfs sein Zeugnis der gewaltig Herausforderung -- vor die wir seit dem Fall von Mauer und Stacheldraht im November 1989 stellen sein -- es gehen jetzt darum -- die Folge von 45 Jahr katastrophal sozialistisch Misswirtschaft zu beseitigen -- zugleich müssen wir uns den gestiegen international Anforderung an ein wied Rvereinigtes Deutschland stellen -- dieses Zusammentreffen gewaltig Aufgabe erfordern den Zusammenhalt aller Kraft -- und ohne die finanz- und haushaltspolitisch Absicherung sein der Einigungsprozeß nicht gestaltbar -- die Spd unterstellen uns -- wir haben die Problem im Zusammenhang mit der Wiedervereinigung verharmlosen und den Bür

In [48]:
sentences = nltk.word_tokenize(filtered_data.text_clean[0])
sentences

['Frau',
 'Präsidentin',
 '!',
 'Meine',
 'sehr',
 'geehrten',
 'Damen',
 'und',
 'Herren',
 '!',
 'Der',
 'Ihnen',
 'heute',
 'zur',
 'ersten',
 'Beratung',
 'vorliegende',
 'Entwurf',
 'des',
 'Bundeshaushalts',
 '1991',
 'ist',
 'der',
 'Haushalt',
 'der',
 'Wiedervereinigung',
 '.',
 'Fast',
 'ein',
 'Viertel',
 'aller',
 'Ausgaben',
 'von',
 'über',
 '400',
 'Milliarden',
 'DM',
 'bezieht',
 'sich',
 'auf',
 'die',
 'am',
 '3',
 '.',
 'Oktober',
 '1990',
 'neu',
 'hinzugekommenen',
 'Bundesländer',
 '.',
 'Umfang',
 'und',
 'Struktur',
 'dieses',
 'Haushaltsentwurfs',
 'sind',
 'Zeugnis',
 'der',
 'gewaltigen',
 'Herausforderungen',
 ',',
 'vor',
 'die',
 'wir',
 'seit',
 'dem',
 'Fall',
 'von',
 'Mauer',
 'und',
 'Stacheldraht',
 'im',
 'November',
 '1989',
 'gestellt',
 'sind',
 '.',
 'Es',
 'geht',
 'jetzt',
 'darum',
 ',',
 'die',
 'Folgen',
 'von',
 '45',
 'Jahren',
 'katastrophaler',
 'sozialistischer',
 'Misswirtschaft',
 'zu',
 'beseitigen',
 '.',
 'Zugleich',
 'müssen',
 

In [50]:
def tokenize(text):
    words = nltk.word_tokenize(text)
    words = [word for word in words if is_word(word) and not is_stop_word(word)]
    return " ".join(words).lower()
        

#filtered_data['text_tokenize'] = filtered_data.text_clean.apply(lemmatize)
filtered_data['text_tokenized'] = filtered_data.text_clean.apply(tokenize)

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_avg_tfidf_scores(data):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data.text_tokenized)
    data['avg_tfidf'] = X.mean(axis=1).ravel().A1
    return data

filtered_data = get_avg_tfidf_scores(filtered_data)